Q1: What proportion of students who answer this question quickly will pass the class?
Assume the probability of passing the class is 0.9.
Assume the probability of answering this question quickly is 0.6 if you'll pass the class,
while the probability drops to 0.3 if you'll not pass the class. 

We want to find the probability that a student will pass the class given that they answer this question quickly, i.e., \(P(\text{Pass} \mid \text{Quick})\).

### Given
- \(P(\text{Pass}) = 0.9\)
- \(P(\text{Not Pass}) = 1 - 0.9 = 0.1\)
- \(P(\text{Quick} \mid \text{Pass}) = 0.6\)
- \(P(\text{Quick} \mid \text{Not Pass}) = 0.3\)

### By Bayes’ Theorem
\[
P(\text{Pass} \mid \text{Quick}) \;=\; 
  \frac{ P(\text{Quick} \mid \text{Pass}) \times P(\text{Pass}) }
       { P(\text{Quick} \mid \text{Pass}) \times P(\text{Pass})
       \;+\; P(\text{Quick} \mid \text{Not Pass}) \times P(\text{Not Pass}) }.
\]

### Plug in the numbers:
\[
\begin{aligned}
P(\text{Pass} \mid \text{Quick})
&= \frac{(0.6)(0.9)}{(0.6)(0.9) + (0.3)(0.1)} \\
&= \frac{0.54}{0.54 + 0.03} \\
&= \frac{0.54}{0.57} \\
&\approx 0.9474 \quad (\text{about } 94.74\%).
\end{aligned}
\]

**Answer: Approximately 94.74% of the students who answer the question quickly will pass the class.**

Q2: 
\begin{align*}
\text{Likelihood} && p(x|\theta,n) &={} \texttt{Multinomial}\\\\
\text{Prior} && p(\theta|\boldsymbol{\alpha}) &={} \texttt{Dirichlet} \\ 
\text{Posterior} && p(\theta|x,n,\boldsymbol{\alpha}) &={} \overset{\text{Bayes Theorem}}{\frac{p(x|\theta)p(\theta)}{p(x)}} = \; \texttt{What?}\\
&&& \quad\quad\quad\quad\quad\; = \; \texttt{What distribution?}\\
&&& \quad\quad\quad\quad\quad\quad\;\;\, \texttt{With what parameterization?}\\
\end{align*}

### _The posterior is always proportional to the joint_

$$\large p(\theta|x) = (1/c)\times p(\theta,x) \propto{} p(\theta,x)$$

**_and the posterior is always a proper distribution if the prior is_**

When you use a Dirichlet prior together with a multinomial likelihood, the Dirichlet is a *conjugate prior* for the multinomial.  Conjugacy means that updating with observed counts \(x\) yields another Dirichlet distribution with “shifted” parameters.  Concretely:

1. **Prior**  
   \[
   p(\boldsymbol{\theta} \mid \boldsymbol{\alpha}) 
   \;=\;
   \mathrm{Dirichlet}\bigl(\alpha_1,\alpha_2,\dots,\alpha_K\bigr)
   \]
   Here, \(\boldsymbol{\alpha} = (\alpha_1,\ldots,\alpha_K)\) are the Dirichlet hyperparameters, and \(\boldsymbol{\theta}\) is the vector of category probabilities for the multinomial.

2. **Likelihood (Multinomial)**  
   \[
   p(\mathbf{x}\mid \boldsymbol{\theta},n) 
   \;=\;
   \mathrm{Multinomial}(\mathbf{x} \mid n,\boldsymbol{\theta}),
   \]
   where \(\mathbf{x} = (x_1,x_2,\dots,x_K)\) are the observed counts of each of the \(K\) categories, and \(n = \sum_j x_j\) is the total count.

3. **Posterior**  
   By Bayes’ theorem and the property of conjugacy, the posterior distribution is again a Dirichlet, with parameters
   \[
   \boldsymbol{\alpha}' \;=\;
   (\alpha_1 + x_1,\; \alpha_2 + x_2,\; \dots,\; \alpha_K + x_K).
   \]
   Thus
   \[
   p(\boldsymbol{\theta}\mid \mathbf{x},n,\boldsymbol{\alpha})
   \;=\;
   \mathrm{Dirichlet}\!\bigl(\alpha_1 + x_1,\;\alpha_2 + x_2,\;\dots,\;\alpha_K + x_K\bigr).
   \]

### Summary

- **What distribution?**  A Dirichlet.  
- **Parameterization?**  The usual “concentration” or “alpha” parameters get incremented by the observed counts.  So if the prior was \(\mathrm{Dirichlet}(\alpha_1,\dots,\alpha_K)\), the posterior is \(\mathrm{Dirichlet}(\alpha_1 + x_1,\dots,\alpha_K + x_K)\).

Q3: Create a sample from a three category multinomial distribution and visualize the multinomial distribution and the multinomial sample; then, visualize the simplex Dirichlet distribution of the prior and the posterior using a ternary plot.

In [1]:
import numpy as np
from scipy import stats
import plotly.figure_factory as ff

# ------------------------------------------------------------------------------
# Create a structured grid over the 3D simplex (p1 + p2 + p3 = 1)
# ------------------------------------------------------------------------------
num_points = 100
p1_lin = np.linspace(0, 1, num_points)       # Step from 0 to 1
p2_lin = 1 - p1_lin.copy()                  # Tied to p1 so p1+p2 <= 1
p_     = np.linspace(0, 1, num_points)      # A second dimension for the "triangle"

# Create a 2D mesh, then flatten it
p1 = (p1_lin[np.newaxis, :] * p_[:, np.newaxis]).flatten() 
p2 = (p2_lin[np.newaxis, :] * p_[:, np.newaxis]).flatten()
p3 = 1 - p1 - p2                             # p3 so that p1 + p2 + p3 = 1
p  = np.c_[p1, p2, p3]                       # Shape: (num_points^2, 3)

# ------------------------------------------------------------------------------
# Define a slightly "non-flat" Dirichlet prior, compute PDF on the grid
# ------------------------------------------------------------------------------
alpha_prior = np.array([1.1, 1.1, 1.1])      # Avoid a perfectly flat distribution
dirichlet_prior = stats.dirichlet(alpha=alpha_prior)
dirichlet_prior_densities = dirichlet_prior.pdf(p.T)

# ------------------------------------------------------------------------------
# Create the ternary contour figure
# ------------------------------------------------------------------------------
fig = ff.create_ternary_contour(
    p.T,                               # Array of shape (3, N)
    dirichlet_prior_densities,         # PDF values, shape (N,)
    pole_labels=["p1", "p2", "p3"],    # Labels for corners of the simplex
    interp_mode="cartesian",           # Must be 'cartesian' for contouring
    showscale=True                     # Add color scale
)

fig.update_layout(
    title="Ternary Contour Plot of Dirichlet(1.1, 1.1, 1.1)",
    width=700,
    height=600
)

# Show (use 'png' to inline in some environments)
fig.show("png")



ModuleNotFoundError: No module named 'plotly'